<a href="https://colab.research.google.com/github/duanchi1230/NLP_Project_AI2_Reasoning_Challenge/blob/master/LSH_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Set up

In [0]:
!pip install hnswlib
!pip install transformers

     |████████████████████████████████| 296kB 36.2MB/s 
  ERROR: Failed building wheel for hnswlib
  Running setup.py clean for hnswlib
Failed to build hnswlib
    Running setup.py install for hnswlib ... done
     |████████████████████████████████| 573kB 11.5MB/s 
     |████████████████████████████████| 1.0MB 47.2MB/s 
     |████████████████████████████████| 890kB 56.3MB/s 
     |████████████████████████████████| 3.7MB 19.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=842144f87904cf7c8bb2a4fe276fb33779b9fe5adfa0979961a300b99c86604a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from pathlib import Path
import csv
import random
import json


from scipy.special import softmax
import hnswlib
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, BertPreTrainedModel, RobertaModel
from transformers import AdamW

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_root_path = Path('/content/drive/My Drive/dataset/data4lsh/')
data_root_path.mkdir(parents=True, exist_ok=True)

## 1. Hyperparameters

In [0]:
output_dir = Path.cwd() / 'out/'
cache_dir = Path.cwd() / 'cache/'
seed = 2020
transformer_model = 'roberta-base'
use_knowledge=False
use_hash=False
knowledge_refresh_rate = 100 # refresh embedding every embed_refresh_rate steps
weight_decay = 0.01
adam_epsilon = 1e-6
learning_rate = 1e-5
num_train_epochs = 6
eval_batch_size = 32
train_batch_size = 16
accumulate_steps = 2
max_seq_length = 128
mode = 'train'
start_epoch = 1

## 2.0 Define model

In [0]:
ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP = {
    "roberta-base": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-pytorch_model.bin",
    "roberta-large": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-pytorch_model.bin",
    "roberta-large-mnli": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-mnli-pytorch_model.bin",
    "distilroberta-base": "https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-pytorch_model.bin",
    "roberta-base-openai-detector": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-openai-detector-pytorch_model.bin",
    "roberta-large-openai-detector": "https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-openai-detector-pytorch_model.bin",
}
class KnowledgeRoberta(BertPreTrainedModel):
  config_class = RobertaConfig
  pretrained_model_archive_map = ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
  base_model_prefix = "roberta"
  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels
    self.roberta = RobertaModel(config)
    self.classifier = nn.Sequential(
        nn.Dropout(config.hidden_dropout_prob),
        nn.Linear(config.hidden_size * 2, config.hidden_size * 2),
        nn.Tanh(),
        nn.Dropout(config.hidden_dropout_prob),
        nn.Linear(config.hidden_size * 2, config.num_labels)
    )
    self.lsh_hash = None
    self.knowledge_vecs = None # shape: (n_facts, hidden_size)

  def _retrieve_knowledge(self, cls_vec, k=10):    
    if use_hash:
      labels, distances = self.lsh_hash.knn_query(cls_vec, k=k)
      distances = torch.as_tensor(softmax(1-distances, axis=1))
      adj_vecs = self.lsh_hash.get_items(labels.reshape(-1))
      adj_vecs = np.asarray(adj_vecs)
      adj_vecs = torch.from_numpy(np.sum(adj_vecs.reshape(k, cls_vec.size(0), -1), axis=1)).float()
      knowledge_attention_vec = distances.matmul(adj_vecs)  
    else:
      attention_weights = torch.matmul(cls_vec, torch.transpose(self.knowledge_vecs, 0, 1))
      knowledge_attention_vec = torch.matmul(attention_weights, self.knowledge_vecs)
    return knowledge_attention_vec
    

  def forward(
      self,
      input_ids=None,
      attention_mask=None,
      token_type_ids=None,
      position_ids=None,
      head_mask=None,
      inputs_embeds=None,
      labels=None,
  ):

    outputs = self.roberta(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        head_mask=head_mask,
        inputs_embeds=inputs_embeds,
    )
    cls_vec = outputs[0][:,0,:] # [CLS] token output
    knowledge_vec = self._retrieve_knowledge(cls_vec.detach().cpu()).to(device)

    logits = self.classifier(torch.cat((cls_vec, knowledge_vec), dim=1))
    outputs = (logits,) + outputs[2:]
    if labels is not None:
      if self.num_labels == 1:
        #  We are doing regression
        loss_fct = nn.MSELoss()
        loss = loss_fct(logits.view(-1), labels.view(-1))
      else:
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
      outputs = (loss,) + outputs

    return outputs  # (loss), logits, (hidden_states), (attentions)

## 2. Main code

In [0]:
# make output directory and cache directory
output_dir.mkdir(exist_ok=True)
cache_dir.mkdir(exist_ok=True)

# Save file names for both training and testing
output_model_file = output_dir / "model.bin"
output_config_file = output_dir / 'config.bin'

# device information
device = torch.device("cuda")

# Set random seeds
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def get_tokenizer():
  tokenizer = RobertaTokenizer.from_pretrained(transformer_model)
  return tokenizer


def get_model(tokenizer_len=None):
  if use_knowledge:
    model_class = KnowledgeRoberta
  else:
    model_class = RobertaForSequenceClassification

  if mode == 'train' or mode == 'test_without_train':
    model = model_class.from_pretrained(
        transformer_model, cache_dir=cache_dir, num_labels=5)
    if tokenizer_len is not None:
      model.resize_token_embeddings(tokenizer_len)
  elif mode == 'test' or mode == 'continue_train':
    model = model_class(
        RobertaConfig.from_json_file(output_config_file))
    model.load_state_dict(torch.load(output_model_file))
  else:
    raise NotImplementedError(
        f'No such mode called {mode}, error raised from get_model.')
  return model.to(device)


def get_optimizer(model):
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(
          nd in n for nd in no_decay)], 'weight_decay': weight_decay},
      {'params': [p for n, p in model.named_parameters() if any(
          nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]
  optimizer = AdamW(optimizer_grouped_parameters,
                    lr=learning_rate, eps=adam_epsilon)
  return optimizer


def convert_string_to_ids(tokenizer, strings):
  token_info = tokenizer.batch_encode_plus(
      strings,
      max_length=max_seq_length,
      pad_to_max_length=True,
      return_token_type_ids=True,
      return_attention_masks=True,
      return_tensors='pt')
  return token_info['input_ids'], token_info['attention_mask'], token_info['token_type_ids']


def load_dataset(train_test_dev, tokenizer):
  input_texts, labels = [], []
  path = data_root_path / (train_test_dev + '.jsonl')
  with open(path, 'r') as f:
    for line in f:
      d = json.loads(line)
      input_texts.append(d['input_text'])
      labels.append(d['target_text'])

  input_ids, input_attention_mask, token_type_ids = convert_string_to_ids(tokenizer, input_texts)
  labels = torch.tensor(labels, dtype=torch.long)
  dataset = TensorDataset(input_ids, input_attention_mask, token_type_ids, labels)
  if train_test_dev == 'train':
    return DataLoader(dataset, shuffle=True, batch_size=train_batch_size, pin_memory=True)
  else:
    return DataLoader(dataset, shuffle=False, batch_size=eval_batch_size)

def build_index(model, tokenizer):
  model.lsh_hash = None

  dim = 10
  num_elements = 100

  # Generating sample data
  path = data_root_path / 'openbook_corpus_combined.txt'
  with open(path, 'r') as f:
    input_ids, attention_mask, token_type_ids = convert_string_to_ids(tokenizer, f.readlines())
  dataset = TensorDataset(input_ids, attention_mask, token_type_ids)
  dataloader = DataLoader(dataset, shuffle=False, batch_size=eval_batch_size)

  base_model = model.roberta
  data = []
  with torch.no_grad():
    for input_ids, attention_mask, token_type_ids in dataloader:
      
      outputs = base_model(input_ids=input_ids.to(device),
                          attention_mask=attention_mask.to(device),
                          token_type_ids=token_type_ids.to(device))
      data.append(outputs[0][:,0,:].cpu())

  data = torch.cat(data)
  data_labels = torch.arange(data.size(0))

  if use_hash:
    # Declaring index
    p = hnswlib.Index(space='ip', dim=data.size(1)) # possible options are l2, cosine or ip

    # Initing index - the maximum number of elements should be known beforehand
    p.init_index(max_elements=data.size(0), ef_construction = 200, M = 64)

    # Element insertion (can be called several times):
    p.add_items(data, data_labels)

    # Controlling the recall by setting ef:
    p.set_ef(50) # ef should always be > k

    # return p
    model.lsh_hash = p

  else:
    model.knowledge_vecs = data



def main():
  # Get tokenizer
  tokenizer = get_tokenizer()

  # Create model
  model = get_model()

  # Training phase
  if mode == 'train' or mode == 'continue_train':
    # Prepare train set
    train_dataloader = load_dataset('train', tokenizer)
    dev_dataloader = load_dataset('dev', tokenizer)

    # Prepare optimizer
    optimizer = get_optimizer(model)

    # Start training
    print('Start training...')

    for epoch in range(num_train_epochs):
      model.train()
      # Train for one epoch, and evaluate later
      train_loss = 0
      n_totals_train, n_corrects_train = 0,0
      for step, (input_ids, attention_mask, token_type_ids, labels) in enumerate(train_dataloader):

        if use_knowledge:
          if step % knowledge_refresh_rate == 0:
            model.eval()
            build_index(model, tokenizer)
            model.train()

        outputs = model(input_ids=input_ids.to(device),
                        attention_mask=attention_mask.to(device),
                        token_type_ids=token_type_ids.to(device),
                        labels=labels.to(device))
        
        loss = outputs[0]
        loss.backward()
        if (step + 1) % accumulate_steps == 0 or step + 1 == len(train_dataloader):
          optimizer.step()
          optimizer.zero_grad()
        train_loss += loss.item()

        logits = outputs[1].cpu()
        predictions = torch.argmax(logits, axis=1).squeeze()
        labels = labels.squeeze()
        n_totals_train += len(labels)
        n_corrects_train += torch.eq(predictions, labels).sum().item()

      # eval on dev set
      model.eval()
      eval_loss = 0
      with torch.no_grad():
        n_totals, n_corrects = 0,0
        for input_ids, attention_mask, token_type_ids, labels in dev_dataloader:
          outputs = model(input_ids=input_ids.to(device),
                          attention_mask=attention_mask.to(device),
                          token_type_ids=token_type_ids.to(device),
                          labels=labels.to(device))

          loss = outputs[0]
          eval_loss += loss.item()
 
          logits = outputs[1].cpu()
          predictions = torch.argmax(logits, axis=1).squeeze()
          labels = labels.squeeze()
          n_totals += len(labels)
          n_corrects += torch.eq(predictions, labels).sum().item()

      train_loss = train_loss / n_totals_train
      val_loss = eval_loss / n_totals
      train_acc = n_corrects_train / n_totals_train
      val_acc = n_corrects / n_totals
      print(f'Train epoch {start_epoch + epoch} loss: {train_loss:.3f} acc: {train_acc:.3f}, val loss: {val_loss:.3f}, val acc: {val_acc:.3f}')
    
    
    # # save final model
    # model_to_save = model.module if hasattr(model, 'module') else model
    # torch.save(model_to_save.state_dict(),  output_model_file)
    # # save config file
    # model_to_save.config.to_json_file(output_config_file)
    # return model

In [0]:
main()


Start training...


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Train epoch 1 loss: 0.088 acc: 0.252, val loss: 0.044, val acc: 0.262
Train epoch 2 loss: 0.087 acc: 0.255, val loss: 0.044, val acc: 0.264
Train epoch 3 loss: 0.087 acc: 0.272, val loss: 0.044, val acc: 0.282
Train epoch 4 loss: 0.086 acc: 0.285, val loss: 0.044, val acc: 0.279
Train epoch 5 loss: 0.084 acc: 0.331, val loss: 0.044, val acc: 0.265
Train epoch 6 loss: 0.079 acc: 0.405, val loss: 0.047, val acc: 0.270


 [6492 x 768], m2: [16 x 768]

In [0]:
import hnswlib
import numpy as np

dim = 10
num_elements = 100

# Generating sample data
data = np.float32(np.random.random((num_elements, dim)))
data_labels = np.arange(num_elements)

# Declaring index
p = hnswlib.Index(space = 'ip', dim = dim) # possible options are l2, cosine or ip

# Initing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)

# Element insertion (can be called several times):
p.add_items(data, data_labels)

# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

# Query dataset, k - number of closest elements (returns 2 numpy arrays)
labels, distances = p.knn_query(data[0], k = 1)

In [0]:
print(f'{labels}: {distances}')